In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys

In [ ]:
##
#Define Path to Code Database
DirPath = '/Your/Path/To/Code/'

##
#Define Output Path
OutputPath = '/Your/Path/To/Output/'

In [ ]:
##
#Load Functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from ParameterOptionsSimulation import *

In [ ]:
##
#Read Parameters
opt = ParameterOptionsSimulation()

In [ ]:
##
#Define Motion Profile (Translational Motion = 0.2 mm/s Along Single Axis)
MotionScale = np.ones(opt["nTR"].astype(np.int32)[0])*0.2

In [ ]:
##
#Estimate Signal With & Without Motion

##
#Define Diffusion Coeficient Array (log Scale)
DArr = np.linspace(-6,-1,num=100)

##
#Initialise Signal Arrays 
Signal = np.zeros((int(opt['nTR'][0]),DArr.shape[0]),dtype='c8')
SignalMotion = np.zeros((int(opt['nTR'][0]),DArr.shape[0]),dtype='c8')

##
#Simulate Signals
for idx, D in enumerate(DArr):
    opt['D'] = np.asarray([10**D], dtype='f8')
    Signal[:,idx] = EPGMotion(opt)
    SignalMotion[:,idx] = EPGMotion(opt, MotionScale)

In [ ]:
##
#Characterise Signal Contribution from Different k-States (Longitudinal Contribution)

##
#Define Diffusion Coefficients
DLow = 1E-4
DHigh = 1E-3

##
#Initialise Output Arrays
Sk_Low = np.zeros((11), dtype = 'c8')
Sk_High = np.zeros((11), dtype = 'c8')

##
#Define Constants (b-value and Velocity Equivalent) - Longitudinal Contribution
gamma_tau_G = opt['gamma'] * opt['tau'] * opt['G']
bLong = (gamma_tau_G**2) * opt['TR'] * 10**-21
vLong = gamma_tau_G * opt['TR'] * MotionScale[-1] * 10**-12

##
#Estimate Contribution of Different k-States
for k in range(0,11):
    Sk_Low[k] = np.exp(-bLong[0] * k**2 * DLow)*np.exp(-1j * vLong[0] * k)
    Sk_High[k] = np.exp(-bLong[0] * k**2 * DHigh)*np.exp(-1j * vLong[0] * k)

In [ ]:
##
#Plot 
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(12,8)

##
#Define x-axis

##
#Plot Magnitude Attentuation
axs[0,0].plot(10**DArr, np.abs(SignalMotion[-1,:]/Signal[-1,:]),'#1f77b4',linewidth=2)
axs[0,0].set_xscale('log')

##
#Plot Phase Diference 
axs[0,1].plot(10**DArr,np.angle(SignalMotion[-1,:]/Signal[-1,:]),'#1f77b4',linewidth=2)
axs[0,1].set_xscale('log')


##
#Plot contribution of different k-states - Magnitude
axs[1,0].plot(range(0,11),np.abs(Sk_Low),linewidth=2)
axs[1,0].plot(range(0,11),np.abs(Sk_High),linewidth=2)

##
#Plot contribution of different k-states - Phase
axs[1,1].plot(range(0,11),np.angle(Sk_Low),linewidth=2)
axs[1,1].plot(range(0,11),np.angle(Sk_High),linestyle='--',linewidth=2)

##
#Add labels etc
fig.subplots_adjust(wspace=0.2,hspace=0.4)
axs[0,0].set_ylim([0.2,1.05])
axs[0,1].set_ylim([0,np.pi])
axs[0,0].set_ylabel('Magnitude Attenuation (norm.)')
axs[0,1].set_ylabel('Phase Difference (rad.)')
axs[0,0].set_xlabel(r'D (mm$^2$/s)' )
axs[0,1].set_xlabel(r'D (mm$^2$/s)')
axs[0,0].set_yticks([0.2,0.4,0.6,0.8,1])
axs[0,1].set_yticks([0, np.pi/2, np.pi],[0, r'$\pi$/2', r'$\pi$'])
axs[0,0].set_title('Motion Induced Magnitude Attenuation')
axs[0,1].set_title('Motion Induced Phase Difference')
axs[1,0].set_title(r'Relative Contribution of k states (Magnitude)')
axs[1,1].set_title(r'Relative Contribution of k states (Phase)')
axs[1,0].set_xlabel(r'k/q')
axs[1,1].set_xlabel(r'k/q')
axs[1,0].set_ylabel('Relative Magnitude (norm.)')
axs[1,1].set_ylabel('Relative Phase (rad.)')
axs[1,0].legend([''.join(['D =','{:.1E}'.format(DLow),r' mm$^2$/s']),''.join(['D =','{:.1E}'.format(DHigh),r' mm$^2$/s'])])
axs[1,1].legend([''.join(['D =','{:.1E}'.format(DLow),r' mm$^2$/s']),''.join(['D =','{:.1E}'.format(DHigh),r' mm$^2$/s'])])
axs[1,1].set_yticks([-np.pi, 0, np.pi],[r'$\pi$', 0, r'$\pi$'])
axs[0,0].text(-0.1, 1.10, '(a)', transform=axs[0,0].transAxes, size=15)
axs[0,1].text(-0.1, 1.10, '(b)', transform=axs[0,1].transAxes, size=15)
axs[1,0].text(-0.1, 1.10, '(c)', transform=axs[1,0].transAxes, size=15)
axs[1,1].text(-0.1, 1.10, '(d)', transform=axs[1,1].transAxes, size=15)

In [ ]:
##
#Save Figure
fig.savefig(''.join([OutputPath,'FigureS4.png']),dpi=300,format='png',bbox_inches='tight')